In [4]:
import pandas as pd
import numpy as np

## Import data

In [5]:
# Results folder
folder = '../results/'
data_mm_df = pd.read_csv(folder + 'csv/results_CLM5_mm.csv')
data_pe_df = pd.read_csv(folder + 'csv/results_CLM5_pe.csv')

## Merge Data

In [6]:
# Combine multimarket and partial eq results
data_mm_df['Type'] = 'Multimarket'
data_pe_df['Type'] = 'Partial Equilibrium'
data_merged_df     = pd.concat([data_mm_df, data_pe_df])

In [7]:
# Subset countries
country_subset = ['India', 'United States', 'China', 'Brazil']
data_merged_df = data_merged_df.query('Country in @country_subset')

# Fix crop labels
data_merged_df['Crop'] = data_merged_df['Crop'].apply(lambda x: x.capitalize())

In [65]:
# Aggregate data
data_agg_df = data_merged_df.groupby(['Country', 'Type', 'Year']).sum().reset_index()

# Recompute changes in surplus
data_agg_df['Percent_Change_in_Consumer_Surplus'] = (data_agg_df['Change_in_Consumer_Surplus'] / 
                                                     data_agg_df['Consumer_Surplus_Original'])
data_agg_df['Percent_Change_in_Producer_Surplus'] = (data_agg_df['Change_in_Producer_Surplus'] / 
                                                     data_agg_df['Producer_Surplus_Original'])
data_agg_df['Percent_Change_in_Total_Surplus']    = ((data_agg_df['Change_in_Producer_Surplus'] 
                                                     + data_agg_df['Change_in_Consumer_Surplus']) / 
                                                    (data_agg_df['Producer_Surplus_Original']
                                                     + data_agg_df['Consumer_Surplus_Original']))

## Generate Tables

In [83]:
output_file = '../results/xlsx/results_CLM5_formatted.xlsx'
writer = pd.ExcelWriter(output_file, engine='xlsxwriter')

In [84]:
# Producer and Consumer Surplus Changes
for country in data_merged_df['Country']:
    
    pivot_table_df = (data_agg_df.query('Country == @country')
                     .pivot_table(index = 'Year', columns = ['Type'], 
                                  values = ['Percent_Change_in_Consumer_Surplus', 'Percent_Change_in_Producer_Surplus', 
                                  'Percent_Change_in_Total_Surplus']))
    
    pivot_table_df = pivot_table_df.applymap(lambda x: '{0:.2f}%'.format(x*100))
    
    pivot_table_df.to_excel(writer, sheet_name = 'All_' + country)

In [85]:
# Price and Quantity Changes
for crop in data_merged_df['Crop'].unique():
    for col in ['Percent_Price_Change', 'Percent_Quantity_Change']:
        
        pivot_table_df = (data_merged_df.query('Crop == @crop')
                          .pivot_table(index = 'Year', columns = ['Country', 'Type'], values = col)
                         )
        pivot_table_df = pivot_table_df.applymap(lambda x: '{0:.2f}%'.format(x*100))
        
        pivot_table_df.to_excel(writer, sheet_name = crop + '_' + col)

In [86]:
writer.close()